#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링 
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.





### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/KT_aivle/미니프로젝트_5차/data01_train.csv')
data = data.drop('subject', axis=1)
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
data['Activity_dynamic'] = data['Activity']

dynamic = {'STANDING':0, 'SITTING':0, 'LAYING':0,
           'WALKING':1, 'WALKING_UPSTAIRS':1, 'WALKING_DOWNSTAIRS':1}
data['Activity_dynamic'] = data['Activity_dynamic'].map(dynamic)

In [ ]:
x = data.drop(['Activity','Activity_dynamic'], axis=1)
y1 = data['Activity']
y2 = data['Activity_dynamic']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x, y1, y2, test_size=0.25, random_state=1)
X_train.shape, X_val.shape, y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape

((4410, 561), (1471, 561), (4410,), (1471,), (4410,), (1471,))

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 : 

In [ ]:
# 0 or 1 model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1, max_depth=50, n_estimators=200)
rf.fit(x, y2) # y2 : 0 or 1


[0 0 0 ... 0 1 0]


In [ ]:
pd.concat([X_train, y1_train], axis=1)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
1540,0.289553,-0.048711,-0.068146,-0.394232,-0.290747,-0.139756,-0.387611,-0.310921,-0.204092,-0.277472,...,0.134849,-0.196880,0.053475,-0.448275,0.639345,-0.829114,-0.798440,0.023262,0.142953,WALKING
1684,0.283455,-0.011421,-0.083670,-0.193129,0.013447,-0.246846,-0.238358,-0.035645,-0.243574,0.039014,...,-0.297651,-0.745646,-0.056202,-0.683307,0.946357,-0.851826,-0.878032,0.182503,0.016303,WALKING_DOWNSTAIRS
3155,0.285473,-0.030318,-0.095229,-0.955893,-0.859198,-0.817387,-0.958335,-0.849369,-0.817891,-0.887105,...,-0.488811,-0.830884,0.101912,0.732465,0.250616,0.176789,0.724219,0.396149,0.835087,LAYING
2423,0.211232,-0.012406,-0.079095,-0.258280,0.312215,-0.028039,-0.285914,0.275879,-0.053645,-0.103865,...,-0.381716,-0.725697,0.852575,0.003621,0.818095,0.213208,-0.609873,0.197490,0.277599,WALKING
2185,0.348948,-0.027330,-0.054020,-0.034799,0.041823,-0.273195,-0.125336,0.035754,-0.264309,0.224365,...,0.226637,-0.136643,-0.434451,0.149583,-0.697247,0.761420,-0.803697,0.233063,0.032719,WALKING_UPSTAIRS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,0.379591,-0.053781,-0.037593,0.579153,0.474105,-0.265095,0.544391,0.326989,-0.349462,0.784666,...,-0.239562,-0.581172,-0.434818,-0.171448,-0.881385,-0.849425,-0.860679,0.194430,0.021513,WALKING_DOWNSTAIRS
5192,0.401797,-0.053621,-0.113978,0.014104,-0.029484,-0.527964,-0.089131,0.043666,-0.507814,0.280816,...,-0.577141,-0.867323,-0.777464,-0.195736,0.960012,0.549612,-0.683727,0.313537,-0.019291,WALKING_UPSTAIRS
3980,0.259130,-0.016490,-0.108074,-0.986086,-0.974310,-0.967780,-0.988795,-0.974780,-0.968121,-0.922559,...,-0.093890,-0.519162,0.095095,-0.194093,0.124918,0.289303,0.367361,-0.524099,-0.445755,LAYING
235,0.269344,-0.032164,-0.120304,-0.364150,0.021633,-0.255305,-0.394755,0.046830,-0.250063,-0.298838,...,0.322276,0.118263,-0.137159,0.370359,-0.899965,-0.964398,-0.580155,0.338105,0.180816,WALKING


In [ ]:
x.loc[idx_0]

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.032207,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.246705,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,0.388765,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755
6,0.121465,-0.031902,-0.005196,-0.152198,-0.113104,-0.239423,-0.202401,-0.164698,-0.247099,0.114668,...,-0.127764,-0.371055,-0.775779,0.445206,-0.003487,-0.940185,0.041387,-0.886603,0.173338,-0.005627
8,0.284338,0.021956,-0.006925,-0.980153,-0.838394,-0.782357,-0.983683,-0.816199,-0.743923,-0.914011,...,-0.194271,-0.052863,-0.400197,0.021212,-0.009465,-0.282762,0.563343,-0.782072,0.242834,-0.025285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,0.268128,0.006551,-0.102443,-0.994445,-0.955041,-0.957838,-0.995514,-0.950148,-0.955473,-0.929516,...,-0.268783,-0.346015,-0.730052,0.041368,0.133130,-0.075273,0.096583,-0.901800,0.029027,-0.001582
1464,0.299498,-0.012221,-0.114990,-0.981517,-0.952031,-0.967395,-0.981488,-0.946526,-0.962790,-0.921084,...,0.115979,-0.591056,-0.857788,-0.019692,0.283370,-0.272158,0.300582,0.413167,-0.274525,-0.716212
1465,0.295076,-0.008439,-0.110448,-0.446681,-0.162954,-0.232261,-0.466149,-0.194852,-0.265123,-0.356988,...,0.112859,-0.378519,-0.654693,-0.241888,-0.740507,0.958570,-0.117069,-0.508372,0.290134,0.301959
1467,0.268044,-0.014861,-0.105414,-0.990687,-0.976967,-0.981729,-0.991500,-0.977235,-0.981998,-0.938679,...,0.344786,-0.484983,-0.842352,0.190303,0.223932,-0.007977,-0.050694,-0.363524,-0.289190,-0.182467


#### 2) 알고리즘2 : 

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
data_0 = data[(data['Activity']=='LAYING')|(data['Activity']=='SITTING')|(data['Activity']=='STANDING')]

In [ ]:
x = data_0.drop('Activity', axis=1)
y = data_0['Activity']

from sklearn.model_selection import train_test_split
X0_train, X0_val, y0_train, y0_val = train_test_split(x, y, test_size=0.25, random_state=1)

# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=1, max_depth=50, n_estimators=200)
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_val)
# from sklearn.metrics import *
# print(accuracy_score(y_val, y_pred))
# print(classification_report(y_val, y_pred))

In [ ]:
# from lightgbm import LGBMClassifier
# lgmb = LGBMClassifier(random_state=1)
# X_train = np.array(X_train)
# y_train = np.array(y_train)
# X_val = np.array(X_val)
# y_val = np.array(y_val)
# lgmb.fit(X_train, y_train)
# y_pred = lgmb.predict(X_val)
# from sklearn.metrics import *
# print(accuracy_score(y_val, y_pred))
# print(classification_report(y_val, y_pred))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y0_train)
y_val = le.fit_transform(y0_val)

In [ ]:
# from xgboost import XGBClassifier
# xgb = XGBClassifier(random_state=1)
# xgb.fit(X_train, y_train)
# y_pred = xgb.predict(X_val)
# from sklearn.metrics import *
# print(accuracy_score(y_val, y_pred))
# print(classification_report(y_val, y_pred))

* catboost

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_0.iloc[:,:-2] = scaler.fit_transform(data_0.iloc[:,:-2])

<ipython-input-16-36bd650eabed>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0.iloc[:,:-2] = scaler.fit_transform(data_0.iloc[:,:-2])


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.2 MB/s eta 0:00:00


In [ ]:
x = data_0.drop('Activity', axis=1)
y = data_0['Activity']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=1)

In [ ]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier(iterations=1000, learning_rate=0.2, depth=2, loss_function='MultiClass')
cb.fit(X_train, y_train)
y_pred = cb.predict(X_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

0:	learn: 0.8102214	total: 81.4ms	remaining: 1m 21s
1:	learn: 0.6482456	total: 109ms	remaining: 54.2s
2:	learn: 0.5394352	total: 135ms	remaining: 44.7s
3:	learn: 0.4614167	total: 161ms	remaining: 40s
4:	learn: 0.4058737	total: 187ms	remaining: 37.2s
5:	learn: 0.3597812	total: 215ms	remaining: 35.6s
6:	learn: 0.3260951	total: 241ms	remaining: 34.2s
7:	learn: 0.3013212	total: 267ms	remaining: 33.1s
8:	learn: 0.2780487	total: 295ms	remaining: 32.5s
9:	learn: 0.2599627	total: 322ms	remaining: 31.8s
10:	learn: 0.2430070	total: 348ms	remaining: 31.3s
11:	learn: 0.2279216	total: 375ms	remaining: 30.9s
12:	learn: 0.2213767	total: 400ms	remaining: 30.4s
13:	learn: 0.2128579	total: 428ms	remaining: 30.2s
14:	learn: 0.2095354	total: 459ms	remaining: 30.2s
15:	learn: 0.2056236	total: 483ms	remaining: 29.7s
16:	learn: 0.2009046	total: 512ms	remaining: 29.6s
17:	learn: 0.1983491	total: 538ms	remaining: 29.4s
18:	learn: 0.1895597	total: 567ms	remaining: 29.3s
19:	learn: 0.1854568	total: 595ms	remaini

* auto ml

In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from pycaret.classification import *
clf = setup(data=data_0, target='Activity', use_gpu=True)

,Description,Value
0,Session id,5185
1,Target,Activity
2,Target type,Multiclass
3,Target mapping,"LAYING: 0, SITTING: 1, STANDING: 2"
4,Original data shape,"(3234, 563)"
5,Transformed data shape,"(3234, 563)"
6,Transformed train set shape,"(2263, 563)"
7,Transformed test set shape,"(971, 563)"
8,Numeric features,562
9,Preprocess,True


In [ ]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
best_model_0 = compare_models(n_select=8, include=['lr','knn','nb','svm','rf','xgboost','lightgbm','catboost'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9841,0.9993,0.9841,0.9844,0.9841,0.9761,0.9763,11.2330
lightgbm,Light Gradient Boosting Machine,0.9823,0.9992,0.9823,0.9826,0.9823,0.9735,0.9736,17.7050
xgboost,Extreme Gradient Boosting,0.9792,0.9990,0.9792,0.9796,0.9792,0.9688,0.9690,2.0480
rf,Random Forest Classifier,0.9704,0.9976,0.9704,0.9710,0.9704,0.9556,0.9559,2.9180
lr,Logistic Regression,0.9699,0.9971,0.9699,0.9703,0.9699,0.9549,0.9551,3.4280
svm,SVM - Linear Kernel,0.9629,0.0000,0.9629,0.9650,0.9628,0.9443,0.9454,0.4990
knn,K Neighbors Classifier,0.9240,0.9833,0.9240,0.9262,0.9236,0.8859,0.8873,0.9710
nb,Naive Bayes,0.7040,0.8929,0.7040,0.7399,0.6936,0.5559,0.5802,0.6870


Processing:   0%|          | 0/44 [00:00<?, ?it/s]

In [ ]:
best_model_0[0]

In [ ]:
best_model_0[0].get_all_params()

{'nan_mode': 'Min',
 'gpu_ram_part': 0.95,
 'eval_metric': 'MultiClass',
 'iterations': 1000,
 'leaf_estimation_method': 'Newton',
 'observations_to_bootstrap': 'TestOnly',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'devices': '-1',
 'eval_fraction': 0,
 'pinned_memory_bytes': '104857600',
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'gpu_cat_features_storage': 'GpuRam',
 'fold_size_loss_normalization': False,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'use_best_model': False,
 'meta_l2_frequency': 0,
 'class_names': [0, 1, 2],
 'random_seed': 5185,
 'depth': 6,
 'border_count': 32,
 'min_fold_size': 100,
 'data_partition': 'DocParallel',
 'bagging_temperature': 1,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'leaf_estimation_backtracking': 'AnyImpr

In [ ]:
tuned_model_0 = tune_model(best_model_0[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9892,1.0000,0.9892,0.9894,0.9892,0.9838,0.9839
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9946,1.0000,0.9946,0.9947,0.9946,0.9918,0.9919
4,0.9892,0.9999,0.9892,0.9895,0.9892,0.9837,0.9839
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9946,1.0000,0.9946,0.9947,0.9946,0.9919,0.9919
7,0.9838,0.9989,0.9838,0.9840,0.9838,0.9755,0.9757
8,0.9892,1.0000,0.9892,0.9893,0.9892,0.9837,0.9838


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
data_1 = data[(data['Activity']=='WALKING')|(data['Activity']=='WALKING_UPSTAIRS')|(data['Activity']=='WALKING_DOWNSTAIRS')]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_1.iloc[:,:-2] = scaler.fit_transform(data_1.iloc[:,:-2])

In [ ]:
x = data_1.drop('Activity', axis=1)
y = data_1['Activity']

from sklearn.model_selection import train_test_split
X1_train, X1_val, y1_train, y1_val = train_test_split(x, y, test_size=0.25, random_state=1)

In [ ]:
# from catboost import CatBoostClassifier
# cb = CatBoostClassifier(iterations=10, learning_rate=0.2, depth=2, loss_function='MultiClass')
# cb.fit(X_train, y_train)
# y_pred = cb.predict(X_val)
# print(accuracy_score(y_val, y_pred))
# print(classification_report(y_val, y_pred))

In [ ]:
from pycaret.classification import *
clf = setup(data=data_1, target='Activity', use_gpu=True)

,Description,Value
0,Session id,8301
1,Target,Activity
2,Target type,Multiclass
3,Target mapping,"WALKING: 0, WALKING_DOWNSTAIRS: 1, WALKING_UPSTAIRS: 2"
4,Original data shape,"(2647, 563)"
5,Transformed data shape,"(2647, 563)"
6,Transformed train set shape,"(1852, 563)"
7,Transformed test set shape,"(795, 563)"
8,Numeric features,562
9,Preprocess,True


In [ ]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
best_model_1 = compare_models(n_select=8, include=['lr','et','rbfsvm','xgboost','lightgbm','rf','svm','catboost'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9957,1.0000,0.9957,0.9957,0.9957,0.9935,0.9935,1.5960
lightgbm,Light Gradient Boosting Machine,0.9957,0.9998,0.9957,0.9957,0.9957,0.9935,0.9935,19.6520
et,Extra Trees Classifier,0.9935,0.9998,0.9935,0.9936,0.9935,0.9902,0.9903,1.5890
svm,SVM - Linear Kernel,0.9930,0.0000,0.9930,0.9932,0.9930,0.9894,0.9896,0.2790
catboost,CatBoost Classifier,0.9930,0.9999,0.9930,0.9931,0.9930,0.9894,0.9895,11.4600
xgboost,Extreme Gradient Boosting,0.9914,0.9997,0.9914,0.9915,0.9913,0.9870,0.9871,2.2400
rf,Random Forest Classifier,0.9779,0.9990,0.9779,0.9781,0.9779,0.9667,0.9668,2.8170
rbfsvm,SVM - Radial Kernel,0.9493,0.9958,0.9493,0.9517,0.9494,0.9236,0.9246,3.9800


Processing:   0%|          | 0/44 [00:00<?, ?it/s]

In [ ]:
best_model_1[0]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=8301, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
tuned_model_1 = tune_model(best_model_1[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9946,0.9999,0.9946,0.9947,0.9946,0.9919,0.9919
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9946,1.0000,0.9946,0.9947,0.9946,0.9919,0.9919
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
# CatBoost optuna

import optuna

# 이제 CatBoost 분류기를 정의하고 하이퍼 파라미터를 설정합니다.
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
        'random_strength': trial.suggest_int('random_strength', 1, 100),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'task_type': 'GPU', # GPU 환경이 있다면 GPU 사용
        'eval_metric': 'AUC',
        'verbose': False,
        'random_state': 0
    }

    clf = cb.CatBoostClassifier(**param)
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=20, verbose=False)

    y_pred = clf.predict(X_test)
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)

    return accuracy


# 이제 objective 함수를 사용하여 Optuna를 설정하고 최적화를 실행합니다.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



# 이제 최적화된 하이퍼 파라미터를 사용하여 모델을 학습시키고 예측을 수행할 수 있습니다.
param = study.best_trial.params
clf = cb.CatBoostClassifier(**param)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

* not tune

In [ ]:
best_model_0[0].fit(X0_train, y0_train)
best_model_1[0].fit(X1_train, y1_train)

y0_pred = best_model_0[0].predict(X0_val)
y1_pred = best_model_1[0].predict(X1_val)

In [ ]:
y0_pred = y0_pred.reshape(-1)
y1_pred = y1_pred.reshape(-1)

y_pred = np.concatenate([y0_pred, y1_pred])

In [ ]:
y_val = np.concatenate([y0_val, y1_val])

In [ ]:
print(classification_report(y_val, y_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       266
           SITTING       0.96      0.96      0.96       264
          STANDING       0.96      0.96      0.96       279
           WALKING       1.00      1.00      1.00       256
WALKING_DOWNSTAIRS       0.99      0.99      0.99       203
  WALKING_UPSTAIRS       0.99      1.00      0.99       203

          accuracy                           0.98      1471
         macro avg       0.98      0.98      0.98      1471
      weighted avg       0.98      0.98      0.98      1471



* tune

In [ ]:
tuned_model_0.fit(X0_train, y0_train)
tuned_model_1.fit(X1_train, y1_train)

y0_pred = tuned_model_0.predict(X0_val)
y1_pred = tuned_model_1.predict(X1_val)


y0_pred = y0_pred.reshape(-1)
y1_pred = y1_pred.reshape(-1)

y_pred = np.concatenate([y0_pred, y1_pred])

y_val = np.concatenate([y0_val, y1_val])

print(classification_report(y_val, y_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       266
           SITTING       0.96      0.96      0.96       264
          STANDING       0.96      0.96      0.96       279
           WALKING       1.00      1.00      1.00       256
WALKING_DOWNSTAIRS       1.00      1.00      1.00       203
  WALKING_UPSTAIRS       1.00      1.00      1.00       203

          accuracy                           0.99      1471
         macro avg       0.99      0.99      0.99      1471
      weighted avg       0.99      0.99      0.99      1471



* Deep Learning

In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(tf.__version__)


2.12.0


In [ ]:
import tensorflow as tf

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPool1D, Dropout
# from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
X0_train = np.array(X0_train)
y0_train = np.array(y0_train)

In [ ]:
X0_train.shape, y0_train.shape

((2425, 562), (2425,))

In [ ]:
np.unique(y0_train)

array(['LAYING', 'SITTING', 'STANDING'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y0_train = le.fit_transform(y0_train)
y0_val = le.transform(y0_val)

In [ ]:
il = tf.keras.layers.Input(shape=(562,))
hl = tf.keras.layers.Dense(63, activation='relu')(il)
hl = tf.keras.layers.Dropout(0.2)(hl)
hl = tf.keras.layers.Dense(32, activation='relu')(hl)
hl = tf.keras.layers.Dropout(0.2)(hl)
hl = tf.keras.layers.Dense(16, activation='relu')(hl)
hl = tf.keras.layers.Dropout(0.2)(hl)
ol = tf.keras.layers.Dense(3, activation='softmax')(hl)

model = tf.keras.models.Model(il, ol)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
model.fit(X0_train, y0_train, validation_split=0.2, epochs=100, callbacks=[es])

Epoch 1/100
61/61 [==============================] - 2s 7ms/step - loss: 0.9172 - accuracy: 0.5263 - val_loss: 0.6204 - val_accuracy: 0.6825
Epoch 2/100
61/61 [==============================] - 0s 5ms/step - loss: 0.5978 - accuracy: 0.7139 - val_loss: 0.4415 - val_accuracy: 0.7237
Epoch 3/100
61/61 [==============================] - 0s 4ms/step - loss: 0.4877 - accuracy: 0.7773 - val_loss: 0.3638 - val_accuracy: 0.8557
Epoch 4/100
61/61 [==============================] - 0s 5ms/step - loss: 0.4402 - accuracy: 0.8015 - val_loss: 0.3307 - val_accuracy: 0.8990
Epoch 5/100
61/61 [==============================] - 0s 7ms/step - loss: 0.3741 - accuracy: 0.8381 - val_loss: 0.2261 - val_accuracy: 0.9134
Epoch 6/100
61/61 [==============================] - 0s 7ms/step - loss: 0.3356 - accuracy: 0.8572 - val_loss: 0.2643 - val_accuracy: 0.8887
Epoch 7/100
61/61 [==============================] - 0s 8ms/step - loss: 0.3156 - accuracy: 0.8665 - val_loss: 0.1923 - val_accuracy: 0.9423
Epoch 8/100
6

In [ ]:
pred = model.predict(X0_val)
pred

26/26 [==============================] - 0s 2ms/step


array([[9.9999774e-01, 2.2561142e-06, 1.2014718e-12],
       [4.3886625e-06, 3.3273447e-02, 9.6672219e-01],
       [9.4326795e-04, 9.6916080e-01, 2.9895997e-02],
       ...,
       [1.7755512e-07, 8.0802459e-03, 9.9191952e-01],
       [9.9999940e-01, 5.5011731e-07, 2.4237750e-14],
       [1.0032170e-06, 2.2695275e-02, 9.7730368e-01]], dtype=float32)

In [ ]:
y0_pred = pred.argmax(axis=1)

In [ ]:
print(classification_report(y0_val, y0_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           1       0.90      0.94      0.92       264
           2       0.95      0.90      0.92       279

    accuracy                           0.95       809
   macro avg       0.95      0.95      0.95       809
weighted avg       0.95      0.95      0.95       809



* CNN

In [ ]:
X0_train = np.expand_dims(X0_train, axis=2)

In [ ]:
X0_train.shape

(2425, 562, 1)

In [ ]:
import tensorflow as tf

# 입력층
input_layer = tf.keras.layers.Input(shape=(562, 1))

# 첫번째 convolutional layer
conv1 = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(input_layer)
pool1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)

# 두번째 convolutional layer
conv2 = tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(pool1)
pool2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)

# 세번째 convolutional layer
conv3 = tf.keras.layers.Conv1D(filters=16, kernel_size=3, padding='same', activation='relu')(pool2)
pool3 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv3)

# 완전연결층
flatten = tf.keras.layers.Flatten()(pool3)
dense1 = tf.keras.layers.Dense(64, activation='relu')(flatten)
dense2 = tf.keras.layers.Dense(32, activation='relu')(dense1)

# 출력층
output_layer = tf.keras.layers.Dense(3, activation='softmax')(dense2)

# 모델 구성 및 컴파일
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X0_train, y0_train, validation_split=0.2, epochs=100, callbacks=[es])

Epoch 1/100
61/61 [==============================] - 5s 16ms/step - loss: 0.9037 - accuracy: 0.5923 - val_loss: 0.4160 - val_accuracy: 0.8433
Epoch 2/100
61/61 [==============================] - 0s 7ms/step - loss: 0.3121 - accuracy: 0.8619 - val_loss: 0.2408 - val_accuracy: 0.8887
Epoch 3/100
61/61 [==============================] - 0s 6ms/step - loss: 0.2079 - accuracy: 0.9093 - val_loss: 0.1659 - val_accuracy: 0.9361
Epoch 4/100
61/61 [==============================] - 0s 5ms/step - loss: 0.1560 - accuracy: 0.9381 - val_loss: 0.1425 - val_accuracy: 0.9464
Epoch 5/100
61/61 [==============================] - 0s 5ms/step - loss: 0.1222 - accuracy: 0.9531 - val_loss: 0.0907 - val_accuracy: 0.9670
Epoch 6/100
61/61 [==============================] - 0s 6ms/step - loss: 0.0920 - accuracy: 0.9649 - val_loss: 0.0789 - val_accuracy: 0.9711
Epoch 7/100
61/61 [==============================] - 0s 5ms/step - loss: 0.0798 - accuracy: 0.9716 - val_loss: 0.0779 - val_accuracy: 0.9732
Epoch 8/100


In [ ]:
pred = model.predict(X0_val)
y0_pred = pred.argmax(axis=1)
print(classification_report(y0_val, y0_pred))

26/26 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           1       0.94      0.96      0.95       264
           2       0.96      0.94      0.95       279

    accuracy                           0.97       809
   macro avg       0.97      0.97      0.97       809
weighted avg       0.97      0.97      0.97       809



* RNN

In [ ]:
il = tf.keras.layers.Input(shape=(562, 1))

hl = tf.keras.layers.SimpleRNN(units=64, return_sequences=True)(il)
hl = tf.keras.layers.Dropout(0.2)(hl)

hl = tf.keras.layers.SimpleRNN(units=32)(hl)
hl = tf.keras.layers.Dropout(0.2)(hl)

ol = tf.keras.layers.Dense(units=3, activation='softmax')(hl)

model = tf.keras.models.Model(inputs=il, outputs=ol)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X0_train, y0_train, validation_split=0.2, epochs=100, callbacks=[es])

Epoch 1/100
61/61 [==============================] - 77s 1s/step - loss: 0.6170 - accuracy: 0.6789 - val_loss: 0.3877 - val_accuracy: 0.8784
Epoch 2/100
61/61 [==============================] - 73s 1s/step - loss: 0.3935 - accuracy: 0.8139 - val_loss: 0.3062 - val_accuracy: 0.8990
Epoch 3/100
61/61 [==============================] - 69s 1s/step - loss: 0.4249 - accuracy: 0.7985 - val_loss: 0.4790 - val_accuracy: 0.7732
Epoch 4/100
61/61 [==============================] - 70s 1s/step - loss: 0.5554 - accuracy: 0.7144 - val_loss: 0.3815 - val_accuracy: 0.8309
Epoch 5/100
61/61 [==============================] - 70s 1s/step - loss: 0.4756 - accuracy: 0.7593 - val_loss: 0.3681 - val_accuracy: 0.8515
Epoch 6/100
61/61 [==============================] - 70s 1s/step - loss: 0.4366 - accuracy: 0.7835 - val_loss: 0.3306 - val_accuracy: 0.8742
Epoch 7/100
61/61 [==============================] - 71s 1s/step - loss: 0.4242 - accuracy: 0.7825 - val_loss: 0.3215 - val_accuracy: 0.8969
Epoch 7: earl

In [ ]:
pred = model.predict(X0_val)
y0_pred = pred.argmax(axis=1)
print(classification_report(y0_val, y0_pred))

26/26 [==============================] - 3s 113ms/step
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       266
           1       0.95      0.69      0.80       264
           2       0.78      0.97      0.87       279

    accuracy                           0.89       809
   macro avg       0.90      0.89      0.88       809
weighted avg       0.90      0.89      0.88       809



# 최종 정리

## 0 과 1 분류 모델 구축

In [ ]:
x

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.032207,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.246705,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,0.388765,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,0.050888,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.013902,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,0.277194,-0.012389,-0.131974,-0.994046,-0.940578,-0.917337,-0.994261,-0.932830,-0.908088,-0.936219,...,0.052914,-0.305772,-0.690363,-0.034888,-0.261437,-0.391477,-0.877612,-0.912365,0.114009,0.080146
5877,0.191568,0.013328,-0.105174,-0.126969,-0.121729,-0.327480,-0.192523,-0.109923,-0.295286,0.078644,...,0.325461,-0.622617,-0.879215,0.721718,0.623151,0.866858,-0.445660,-0.690278,0.303194,-0.044188
5878,0.267981,-0.018348,-0.107440,-0.991303,-0.989881,-0.990313,-0.992386,-0.988852,-0.991237,-0.936099,...,0.362439,-0.597610,-0.886851,0.060173,0.228739,0.684400,-0.216665,0.620363,-0.437247,-0.571840
5879,0.212787,-0.048130,-0.121001,-0.041373,0.052449,-0.585361,-0.100714,0.023353,-0.554707,0.219814,...,0.031915,0.196921,-0.053556,0.260880,0.551742,-0.943773,-0.862899,-0.718009,0.292856,0.024920


In [ ]:
# 0 or 1 model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1, max_depth=50, n_estimators=200)
rf.fit(x, y2) # y2 : 0 or 1

RandomForestClassifier(max_depth=50, n_estimators=200, random_state=1)

## 0일 때 1일 때 각 분류 모델

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.0 MB/s eta 0:00:00


In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


* 0일 때

In [ ]:
data_0 = data[(data['Activity']=='LAYING')|(data['Activity']=='SITTING')|(data['Activity']=='STANDING')]

In [ ]:
from pycaret.classification import *
clf = setup(data=data_0, target='Activity', use_gpu=True)

models()

,Description,Value
0,Session id,1387
1,Target,Activity
2,Target type,Multiclass
3,Target mapping,"LAYING: 0, SITTING: 1, STANDING: 2"
4,Original data shape,"(3234, 563)"
5,Transformed data shape,"(3234, 563)"
6,Transformed train set shape,"(2263, 563)"
7,Transformed test set shape,"(971, 563)"
8,Numeric features,562
9,Preprocess,True


,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
best_model_0 = compare_models(n_select=1, include=['catboost']) # 'lr','knn','nb','svm','rf','xgboost','lightgbm',

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9801,0.9990,0.9801,0.9805,0.9801,0.9701,0.9703,12.7130


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
best_model_0

In [ ]:
tuned_model_0 = tune_model(best_model_0)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9736,0.9982,0.9736,0.9738,0.9736,0.9603,0.9604
1,0.9868,0.9987,0.9868,0.9873,0.9868,0.9802,0.9804
2,0.9912,0.9999,0.9912,0.9914,0.9912,0.9868,0.9869
3,0.9867,0.9998,0.9867,0.9872,0.9867,0.9801,0.9803
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,0.9513,0.9971,0.9513,0.9540,0.9513,0.9270,0.9283
6,0.9867,0.9997,0.9867,0.9868,0.9867,0.9801,0.9801
7,0.9779,0.9982,0.9779,0.9784,0.9779,0.9668,0.9670
8,0.9823,0.9990,0.9823,0.9825,0.9823,0.9734,0.9736


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


* 1일 때

In [ ]:
data_1 = data[(data['Activity']=='WALKING')|(data['Activity']=='WALKING_UPSTAIRS')|(data['Activity']=='WALKING_DOWNSTAIRS')]

In [ ]:
from pycaret.classification import *
clf = setup(data=data_1, target='Activity', use_gpu=True)

models()

,Description,Value
0,Session id,7046
1,Target,Activity
2,Target type,Multiclass
3,Target mapping,"WALKING: 0, WALKING_DOWNSTAIRS: 1, WALKING_UPSTAIRS: 2"
4,Original data shape,"(2647, 563)"
5,Transformed data shape,"(2647, 563)"
6,Transformed train set shape,"(1852, 563)"
7,Transformed test set shape,"(795, 563)"
8,Numeric features,562
9,Preprocess,True


,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
best_model_1 = compare_models(n_select=1, include=['lr']) # ,'knn','nb','svm','rf','xgboost','lightgbm','catboost'

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9957,1.0000,0.9957,0.9957,0.9957,0.9935,0.9935,1.8200


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
best_model_1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=7046, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
tuned_model_1 = tune_model(best_model_1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9946,0.9999,0.9946,0.9947,0.9946,0.9919,0.9919
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9946,1.0000,0.9946,0.9947,0.9946,0.9918,0.9919
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9946,1.0000,0.9946,0.9947,0.9946,0.9918,0.9919
7,0.9946,1.0000,0.9946,0.9947,0.9946,0.9919,0.9919
8,0.9946,1.0000,0.9946,0.9947,0.9946,0.9919,0.9919


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


## 함수 구축

In [ ]:
def sensor_classifier(X_test):
  # 0 or 1 예측
  y_pred = rf.predict(X_test)

  # 인덱스 추출
  idx_0 = np.where(y_pred==0)
  idx_1 = np.where(y_pred==1)

  X_test_0 = X_test.loc[idx_0]
  X_test_1 = X_test.loc[idx_1]

  # 예측
  y_pred_0 = tuned_model_0.predict(X_test_0) # 0 일 때 Laying, Sitting, Standing 예측하는 모델
  y_pred_1 = tuned_model_1.predict(X_test_1) # 1 일 때 Walking, Walking-Up, Walking-Down 예측하는 모델
  y_pred = np.concatenate(y_pred_0, y_pred_1)

  # 하나의 데이터 프레임으로 합치기
  concat_X_test_0 = pd.concat([X_test_0, y_pred_0], axis=1)
  concat_X_test_1 = pd.concat([X_test_1, y_pred_1], axis=1)

  predicted_X_test = pd.concat([X_test_0, X_test_1], axis=0)

  return predicted_X_test

In [ ]:
def sensor_classifier_1(X_test):
  # 0 or 1 예측
  y_pred = rf.predict(X_test)

  # 인덱스 추출
  idx_0 = np.where(y_pred==0)
  idx_1 = np.where(y_pred==1)

  X_test_0 = X_test.loc[idx_0]
  X_test_1 = X_test.loc[idx_1]

  # 예측
  y_pred_0 = tuned_model_0.predict(X_test_0) # 0 일 때 Laying, Sitting, Standing 예측하는 모델
  y_pred_1 = tuned_model_1.predict(X_test_1) # 1 일 때 Walking, Walking-Up, Walking-Down 예측하는 모델
  y_pred_0 = pd.DataFrame(index=idx_0, y_pred_0)
  y_pred_1 = pd.DataFrame(index=idx_1, y_pred_1)
  y_pred = pd.concat([y_pred_0, y_pred_1], axis=0)
  y_pred = y_pred.sort_values(index)
  predicted_X_test = pd.concat([X_test, y_pred], axis=1)

  return predicted_X_test

In [ ]:
X_test = pd.read_csv('test.csv')

In [ ]:
X_test

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,0.116695,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.121711,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,0.258587,-0.006958,-0.094029,-0.485825,-0.255627,-0.287639,-0.494420,-0.262824,-0.292375,-0.492052,...,0.439191,0.395960,0.205011,0.449331,-0.357677,0.973246,-0.461628,-0.808655,0.180537,0.125777
2875,0.269073,0.001729,-0.076217,-0.488379,-0.217140,-0.241188,-0.488578,-0.254760,-0.261607,-0.494263,...,0.433308,0.440530,0.218345,0.202008,-0.290002,0.778535,-0.749971,-0.814476,0.176293,0.124005
2876,0.281797,-0.022419,-0.098580,-0.462887,-0.168802,-0.270224,-0.459324,-0.210244,-0.272145,-0.428381,...,0.285612,0.389399,0.128611,-0.098913,-0.898726,0.352131,-0.341420,-0.825102,0.170894,0.118787
2877,0.281635,-0.011462,-0.113489,-0.485487,-0.166245,-0.297803,-0.493246,-0.222799,-0.314319,-0.388461,...,0.116867,0.414409,0.212294,-0.128375,-0.395590,0.496106,0.319388,-0.820620,0.173525,0.120704


In [ ]:
X_test = X_test.iloc[:, :-2]

In [ ]:
predicted_X_test = sensor_classifier(X_test)

CatBoostError: ignored

In [ ]:
X_test = X_test.dropna()

In [ ]:
X_test.rename(columns={'found tBodyAcc-arCoeff()-X,1': 'tBodyAcc-arCoeff()-X1'}, inplace=True)